In [2]:
import pandas as pd 
from sklearn.pipeline import Pipeline
from datacleaning.FetchData import FetchData
from datacleaning.CleanData import CleanData
from datetime import datetime
import plotly.express as px

pd.set_option("display.max_columns", 100)

In [12]:
df = pd.read_csv("data/todays_sessions.csv")
df2 = df[["dcosId", "cumEnergy_Wh", "vehicle_model"]].groupby("dcosId").first().copy()
df2["percentage_energy"] = df2["cumEnergy_Wh"] / df2["cumEnergy_Wh"].sum(axis=0)
df2

,cumEnergy_Wh,vehicle_model,percentage_energy
dcosId,,,
4406,2869.0,e-Golf,0.058958
4407,35518.0,Model 3,0.729892
4410,10275.0,Model 3 Performance,0.211150


In [11]:
df["cumEnergy_Wh"].sum(axis=0)

14014656.0

In [13]:
df

,Unnamed: 0,index,vehicle_maxChgRate_W,peakPower_W,sch_centsPerHr,connectTime,vehicle_model,Duration,userId,regular,Deadline,startChargeTime,sch_centsPerOverstayHr,sch_centsPerKwh,choice,siteId,estCost,DurationHrs,dcosId,lastUpdate,energyReq_Wh,power,stationId,defaultDeadline,scheduled,cumEnergy_Wh,reg_centsPerHr,reqChargeTime_h,finishChargeTime,Overstay,Overstay_h,Time,Power (W)
0,0,0,40000,6327.0,200,2023-03-05 13:15:10,e-Golf,0 days 00:29:07,1060,1,NaN,2023-03-05 13:16:01,300,0,REGULAR,25,1.22791,0.48527,4406,2023-03-05 13:45:08,NaN,"[{'power_W': Decimal('6327'), 'timestamp': Dec...",13,2023-03-05T21:00:00,0,2869.0,150,0.453453,2023-03-05 13:43:13,0 days,0.0,2023-03-05 00:00:00,0.0
1,0,0,40000,6327.0,200,2023-03-05 13:15:10,e-Golf,0 days 00:29:07,1060,1,NaN,2023-03-05 13:16:01,300,0,REGULAR,25,1.22791,0.48527,4406,2023-03-05 13:45:08,NaN,"[{'power_W': Decimal('6327'), 'timestamp': Dec...",13,2023-03-05T21:00:00,0,2869.0,150,0.453453,2023-03-05 13:43:13,0 days,0.0,2023-03-05 00:05:00,0.0
2,0,0,40000,6327.0,200,2023-03-05 13:15:10,e-Golf,0 days 00:29:07,1060,1,NaN,2023-03-05 13:16:01,300,0,REGULAR,25,1.22791,0.48527,4406,2023-03-05 13:45:08,NaN,"[{'power_W': Decimal('6327'), 'timestamp': Dec...",13,2023-03-05T21:00:00,0,2869.0,150,0.453453,2023-03-05 13:43:13,0 days,0.0,2023-03-05 00:10:00,0.0
3,0,0,40000,6327.0,200,2023-03-05 13:15:10,e-Golf,0 days 00:29:07,1060,1,NaN,2023-03-05 13:16:01,300,0,REGULAR,25,1.22791,0.48527,4406,2023-03-05 13:45:08,NaN,"[{'power_W': Decimal('6327'), 'timestamp': Dec...",13,2023-03-05T21:00:00,0,2869.0,150,0.453453,2023-03-05 13:43:13,0 days,0.0,2023-03-05 00:15:00,0.0
4,0,0,40000,6327.0,200,2023-03-05 13:15:10,e-Golf,0 days 00:29:07,1060,1,NaN,2023-03-05 13:16:01,300,0,REGULAR,25,1.22791,0.48527,4406,2023-03-05 13:45:08,NaN,"[{'power_W': Decimal('6327'), 'timestamp': Dec...",13,2023-03-05T21:00:00,0,2869.0,150,0.453453,2023-03-05 13:43:13,0 days,0.0,2023-03-05 00:20:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,2,2,250000,6828.0,200,2023-03-05 19:13:41,Model 3 Performance,0 days 01:35:25,1276,1,NaN,2023-03-05 19:14:42,300,0,REGULAR,25,2.88541,1.59027,4410,2023-03-05 20:50:07,NaN,"[{'power_W': Decimal('17'), 'timestamp': Decim...",15,2023-03-05T20:30:00,0,10275.0,150,1.504833,2023-03-05 20:44:59,0 days,0.0,2023-03-05 23:35:00,0.0
860,2,2,250000,6828.0,200,2023-03-05 19:13:41,Model 3 Performance,0 days 01:35:25,1276,1,NaN,2023-03-05 19:14:42,300,0,REGULAR,25,2.88541,1.59027,4410,2023-03-05 20:50:07,NaN,"[{'power_W': Decimal('17'), 'timestamp': Decim...",15,2023-03-05T20:30:00,0,10275.0,150,1.504833,2023-03-05 20:44:59,0 days,0.0,2023-03-05 23:40:00,0.0
861,2,2,250000,6828.0,200,2023-03-05 19:13:41,Model 3 Performance,0 days 01:35:25,1276,1,NaN,2023-03-05 19:14:42,300,0,REGULAR,25,2.88541,1.59027,4410,2023-03-05 20:50:07,NaN,"[{'power_W': Decimal('17'), 'timestamp': Decim...",15,2023-03-05T20:30:00,0,10275.0,150,1.504833,2023-03-05 20:44:59,0 days,0.0,2023-03-05 23:45:00,0.0
862,2,2,250000,6828.0,200,2023-03-05 19:13:41,Model 3 Performance,0 days 01:35:25,1276,1,NaN,2023-03-05 19:14:42,300,0,REGULAR,25,2.88541,1.59027,4410,2023-03-05 20:50:07,NaN,"[{'power_W': Decimal('17'), 'timestamp': Decim...",15,2023-03-05T20:30:00,0,10275.0,150,1.504833,2023-03-05 20:44:59,0 days,0.0,2023-03-05 23:50:00,0.0
